# Searching for Structure: Parametric Methods

K. Leighly 2017

This lecture was drawn from the following sources:
 - Ivezic 4.4, chapter 6
 - Bishop Chapter 2 & 9
 - G. Richards discussion of hierarchical clustering

## Motivation

Last time we talked about the k-means clustering algorithm, and you had homeork on that topic.

Although it is very easy to apply, in terms of a classification scheme it has some limitations.  
- First, a point has to belong to one and only one cluster.  
- Second, k-means is inherently spherical, as it groups objects by the square distance from the cluster center.

We can address these two limitations using gaussian mixture models.

## Gaussian Mixture Models

First, let us define a **mixture model**.  According to wikipedia, "In statistics, a mixture model is a probabilistic model for representing the presence of subpopulations within an overall population, without requiring that an observed data set should identify the sub-population to which an individual observation belongs."

In simpler terms, you have a set of data, and you are concerned with the presence of groups or clusters in the data set.  You don't have any knowledge about the cluster membership from the data (i.e., no labels).   The mixture model will be able to give you the probability that any give point belongs to any one cluster.  In this way, these types of models address the first point above - each data point is not assigned to one cluster or another, but rather there is a probability derived of the data point belonging to each cluster.  That probability may be essentially one for one cluster, and zero for the others, or the allegence can be split, with significant probabilities that the point belongs to two or more clusters.  In a sense, the mixture models have "soft boundaries" while the kmeans clusters have hard boundaries.

A **gaussian mixture model** is one in which the probability distributions for the points is expressed as a sum of gaussians.  This alleviates the second issue above, since a multi-dimensional gaussian can have a different $\sigma$ in each dimension.

For the development of the Gaussian Mixture Models, we will use Bishop, but note that there is a discussion of mostly the same points in Ivezic 4.4 and 6.3.

Let's first create one dimensional example, i.e., a superposition of three gaussians.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pylab, mlab, pyplot
from scipy import stats
%matplotlib inline

In [ ]:
def gaussian(x, mu, sig):
    return 1./(np.sqrt(2.*np.pi)*sig)*np.exp(-np.power((x - mu)/sig, 2.)/2)

x=np.linspace(0,20,101)

mu1=5.0
mu2=9.0
mu3=13.0
sig1=0.5
sig2=2.0
sig3=1.0
a1=0.3
a2=0.3
a3=1.0-a1-a2

y1=a1*gaussian(x,mu1,sig1)
y2=a2*gaussian(x,mu2,sig2)
y3=a3*gaussian(x,mu3,sig3)
y=y1+y2+y3

In [ ]:
plt.plot(x,y)
plt.plot(x,y1)
plt.plot(x,y2)
plt.plot(x,y3)

import scipy
print scipy.integrate.trapz(y,x)

A Gaussian mixture model has a lot of parameters to be determined. Not only are there the $K$ values of $\mathbf{\mu}_k$ and $\mathbf{\Sigma}_k$, but there is also the mixing coefficients $\mathbf{\pi}_K$. 

Usually, to solve this problem, we would set up a log likelihood, take the derivatives, set them equal to zero, and solve.  But for the above system, we have 8 parameters (because the sums of the amplitude of the normalized gaussians are equal to 1), so that would give 8 coupled equations.  This would be generally difficult to solve.

Instead, we introduce some hidden variables.  If the $M$ Gaussian components are interpreted as different "classes", it means that a particular datum $x_i$ was generated by one and only one of the individual Gaussian components.  We index the components using $j$, which is called the **class label**.  $j$ is a **hidden variable**.  

What does this buy you?  If you knew the class label for each data point, then you could simply split off the data with each class label sequentially, and solve for the $\mu$, $\sigma$ and $a$ for each class one at a time - a practically trivial exercise, knowing what we know about the expectation values for the $\mu$ and $\sigma$ of single gaussians, and also how they are normalized.


To write this more formally, consider a superposition of $K$ gaussians with the form

$$p(\mathbf{x})= \sum_{k=1}^{N} \pi_k \mathcal{N}(\mathbf{x}|\mathbf{\mu}_k,\Sigma_k).$$

Each Gaussian density $\mathcal{N}(\mathbf{x}|\mathbf{\mu}_k,\Sigma_k)$ is called a **component** of the mixture and has its own $\boldsymbol{\mu}_k$ and covariance $\Sigma_k$.

The parameters $\pi_k$ are called the **mixing coefficients**. Since both $p(\mathbf{x})$ and the individual normal distributions are normalized, then

$$\sum_{k=1}^{K} \pi_k=1.$$

Furthermore, $0\leq \pi_k\leq 1$.

Similar to the K-means situation discussion last time, let's introduce a K-dimensional binary random variable $\mathbf{z}$ that is similar to $r_{nk}$ in that it is equal to 1 for one particular $k$, and equal to zero for the others. It is, again, the vector that contains information about cluster membership, and it obeys $\sum_k z_k=1$. 

Let us define the joint distribution the joint distribution $p(\mathbf{x},\mathbf{z})$ in terms of a marginal distribution $p(\mathbf{z})$ and a conditional distribution $p(\mathbf{x}|\mathbf{z})$; we will explore these below.

The marginal distribution $p(\mathbf{z})$ is specified in terms of the mixing coefficients $\pi_k$ such that

$$p(z_k=1)=\pi_k,$$

where $\{\pi_k\}$ satisify $0\leq \pi_k \leq 1$ and $\sum_k\pi_k=1$. 


Remembering that the mixing coefficient $\pi_k$ is the amplitude of the $k$th gaussian, recasting of the problem  in this way makes some sense.

Because $\mathbf{z}$ is either zero or 1, this distribution can also be written

$$p(\mathbf{z}) = \prod_{k=1}^{K} \pi_k^{z_k}.$$

Similarly, the conditional distribution of $\mathbf{x}$ given a particular value of $\mathbf{z}$ is a single Gaussian for component $k$:

$$p(\mathbf{x}|z_k=1) =
\mathcal{N}(\mathbf{x}|\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k),$$

and so for all $k$ components together we have:

$$p(\mathbf{x}|\mathbf{z}) =
\prod_{k=1}^{K}\mathcal{N}(\mathbf{x}|\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k)^{z_k}.$$

We can determine $p(\mathbf{x})$ then by making the joint distribution $p(\mathbf{z})p(\mathbf{x}|\mathbf{z})$, and marginalizing over $\mathbf{z}$, as

$$p(\mathbf{x}) = \sum_z p(\mathbf{z})p(\mathbf{x}|\mathbf{z}). $$

Substituting in for $p(\mathbf{z})$ and $p(\mathbf{x}|\mathbf{z})$ yields

$$p(\mathbf{x})=\sum_{k=1}^{K} \pi_k
\mathcal{N}(\mathbf{x}|\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k)$$

This is the same as the equation written above, so it appears that we have gone around in a circle. The utility of this development is the involvement of the latent or hidden variable $\mathbf{z}$, and we can work with $p(\mathbf{x},\mathbf{z})$, which will be easier to deal with than $p(\mathbf{x})$.

Another useful quantify is the conditional probability of $\mathbf{z}$ given $\mathbf{x}$. 

Let $\gamma(z_k)$ stand for $p(z_k=1|\mathbf{x})$. The value of $\gamma(z_k)$ can be written, via Bayes' theorem as:

$$\gamma(z_k) \equiv p(z_k=1|\mathbf{x}) =
\frac{p(z_k=1)p(\mathbf{x}|z_k=1)}{\sum_{j=1}^K
p(z_j=1)p(\mathbf{x}|z_j=1)}$$

$$=\frac{\pi_k\mathcal{N}(\mathbf{x}|\boldsymbol{\mu}_k,\boldsymbol{\Sigma_k})}{\sum_{j=1}^{K}
\pi_j \mathcal{N}(\mathbf{x}|\boldsymbol{\mu}_j,\boldsymbol{\Sigma}_j)}.$$


What does this give us?  We  can view $\pi_k$ as the prior probability of $z_k=1$, and $\gamma(z_k)$ will be the posterior probability once $\mathbf{x}$ has been observed. 

In addition, $\gamma(z_k)$ can be viewed as the responsibility that component $k$ takes for explaining observation $\mathbf{x}$. It is through this factor that we get the desired "soft" boundaries of clusters that is not present in the k-means analysis. That is, a particular point can "belong" to more than one Gaussian.  Hence, the $\gamma(z_k)$  are called the **responsibilities**.

Another way to say it: each point will have a set of responsibilities corresponding to each of the $k$ Gaussians that will give the probability of belonging to that particular Gaussian.  But unlike the k-means case, each the responsibilities will be between zero and 1 for a single point.

## Example

Let's draw points from the example distribution above, and model it with a Gaussian Mixture Model, using the [scikit-learn algorithm](http://scikit-learn.org/stable/modules/mixture.html).

In [ ]:
mu1=5.0
mu2=9.0
mu3=13.0
sig1=0.5
sig2=2.0
sig3=1.0
a1=0.3
a2=0.3
a3=1.0-a1-a2


data1=np.random.normal(mu1,scale=sig1,size=300)
data2=np.random.normal(mu2,scale=sig2,size=300)
data3=np.random.normal(mu3,scale=sig3,size=400)

temp=np.concatenate((data1,data2,data3),axis=0)
np.random.shuffle(temp)

data=np.zeros([1000,1])
data[:,0]=temp[:]
print type(data)
print data.shape
np.random.shuffle(data)

In [ ]:
pyplot.hist(data,bins=20,histtype='step',normed = True)
plt.plot(x,y)

In [ ]:
from sklearn.mixture import GMM

gmm_temp = GMM(n_components=3)
result=gmm_temp.fit(data)

print 'The derived positions of the three gaussians ',result.means_
print 'The covariances of the three gaussians ',result.covars_
print 'The weights on the three gaussians ',result.weights_
    
predicted=result.predict(data)

score_samples_data=result.score_samples(data)

logprob=score_samples_data[0]
responsibilities=score_samples_data[1]

In [ ]:
plt.plot(data,responsibilities[:,0],'.')
plt.plot(data,responsibilities[:,1],'.')
plt.plot(data,responsibilities[:,2],'.')

In [ ]:
pyplot.hist(data,bins=20,histtype='step')
plt.plot(x,y)
pyplot.hist(data[predicted==0],bins=20,histtype='step')
pyplot.hist(data[predicted==1],bins=20,histtype='step')
pyplot.hist(data[predicted==2],bins=20,histtype='step')

The scikit-learn GMM program has many capabilities not explored here; I urge you to investigate the webpage located [here](http://scikit-learn.org/stable/modules/mixture.html).

### Computing GMM

How do we solve for the gaussian parameter values, as well as the hidden variable $\gamma(z_k)$?

Now we have set up the problem: we have a data set of observations $\{\mathbf{x}_1,\ldots,\mathbf{x}_N\}$, and we will commence to try to solve it. Here, the data set will be represented by an $N \times D$ matrix $\mathbf{X}$ in which there are $N$ points and $D$ dimensions. Correspondingly, the _latent_ variables will be denoted by a $N \times K$ matrix $\mathbf{Z}$, whereby the term latent variable refers to hidden, non-explicit variables in the problem.

We will approach it with our old friend the likelihood.

$$\log
p(\mathbf{X}|\boldsymbol{\pi},\boldsymbol{\mu},\boldsymbol{\Sigma}) =
\sum_{n=1}^N \log \left [\sum_{k=1}^{K} \pi_k
\mathcal{N}(\mathbf{x}_n|\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k)\right ].$$

Before we solve this, we note that while the GMM alleviates the problem of hard cluster boundaries, it has its own problem: the potential presence of **singularities**. These singularities can be easily understood heuristically; they happen when a cluster is identified by a single point. A single point cannot usefully define a width of the associated gaussian; the width will be zero. Since the gaussian function has a term proportional to $1/\sigma$ preceding the exponent, this probability distribution will blow up when one tries to normalize it. This situation doesn't arise in the case of a single Gaussian because multiple points contribute to defining the parameters for a single component. 

Numerically, this can plausibly be dealt with by having the algorithm recognize when a component is collapsing to a single point, and resetting it to a new position with a new width. Alternatively, one could expand in termes of a Bernoulli distribution (the same distribution as a binomial), which turns out to not have this problem with singularities (see  Bishop for details).

## Expectation Maximization Algorithm

Maximizing the log likelihood for GMM is complicated by the fact that, unlike the situation for a single Gaussian, the log is in the summation, and the summation is between the log and the exponent. So we can't use convenient methods to simplify. While there are direct methods to attack this problem, the expectation maximization algorithm converges relatively quickly and works well.

_We will motivate and present the EM algorithm for the Gaussian Mixture model, but note that it can be used for a number of different problems_.  For example, your homework problem 1 due today was essentially to program an expectation maximization algorithm for k-means.  We will also use a EM method for principal components analysis.  Bishop gives some discussion about the generality of this method.  

## EM for Gaussian Mixtures


Recall that the multicomponent Gaussian can be written as:

$$\mathcal{N}(\mathbf{x}|\boldsymbol{\mu},\boldsymbol{\Sigma})=\frac{1}{(2\pi)^{D/2}}\frac{1}{|\boldsymbol{\Sigma}|^{1/2}}
\exp\lbrace -\frac{1}{2}(\mathbf{X}-\boldsymbol{\mu})^T
\boldsymbol{\Sigma}^{-1}(\mathbf{X}-\boldsymbol{\mu})\rbrace $$

where $\boldsymbol{\mu}$ is the D-dimensional mean vector, $\boldsymbol{\Sigma}$ is the $D \times D$ covariance matrix, and $|\boldsymbol{\Sigma}|$ denotes the determinant of $\boldsymbol{\Sigma}$.

So, let's proceed as we usually would, by taking the derivative of the log likelihood $\log p(\mathbf{X}|\boldsymbol{\pi},\boldsymbol{\mu},\boldsymbol{\Sigma})$ with respect to the three variables of interest in turn, and setting them equal to zero. 

### $\boldsymbol{\mu}_k$

We will first start with taking the derivative with respect to $\boldsymbol{\mu}_k$ as if we were finding the best fitting component for that variable. The result is:

$$0=\sum_{n=1}^{N}\frac{\pi_k
\mathcal{N}(\mathbf{x}_n|\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k)}{\sum_j
\pi_j\mathcal{N}(\mathbf{x}_n|\boldsymbol{\mu}_j,\boldsymbol{\Sigma}_j)}
\boldsymbol{\Sigma}_k^{-1} (\mathbf{x}_n-\boldsymbol{\mu}_k).$$

Note that the fraction is the responsibilities, $\gamma(z_{nk})$. 

Multiplying by $\boldsymbol{\Sigma}_k$ and solving for $\boldsymbol{\mu}_k$, we obtain:

$$\boldsymbol{\mu}_k =\frac{1}{N_k} \sum_{k-1}^N \gamma(z_{nk})
\mathbf{x}_n$$

where $N_k$ has been defined as
$$N_k=\sum_{n=1}^N \gamma(z_{nk}).$$

$N_k$ ends up begin a kind of weighted number of points or effective number of points in the $k$th cluster. So, the estimate of $\boldsymbol{\mu}_k$ ends up being a weighted average of the data, where the weights are the responsibilities.  This makes sense for $\boldsymbol{\mu}_k$.

### $\boldsymbol{\Sigma}_k$

Next, take the derivative of the log likelihood with respect to $\boldsymbol{\Sigma}_k$ and set it equal to zero. Following a similar line of reasoning, we find:

$$\boldsymbol{\Sigma}_k = \frac{1}{N_k} \sum_{n=1}^{N} \gamma(z_{nk})
(\mathbf{x}_n - \boldsymbol{\mu}_k) (\mathbf{x}_n
-\boldsymbol{\mu}_k)^T.$$

This has the same form as it would if a single gaussian were being fitted to the data set, but now weighted by the responsibility $\gamma(z_{nk})$.

Note that the product $(\mathbf{x}_n - \boldsymbol{\mu}_k) (\mathbf{x}_n-\boldsymbol{\mu}_k)^T$ is the outer product, not the inner product (which would have the factors reversed).  To remind you, [here is the wikipedia page on the outer product](https://en.wikipedia.org/wiki/Outer_product).

### $\pi_k$

Finally, maximize the log likelihood with respect to the mixing coefficients $\pi_k$. As noted above, the mixing coefficients must sum to 1. This can be achieved by using a _Lagrange multiplier_ and maximizing the following

$$\log
p(\mathbf{X}|\boldsymbol{\pi},\boldsymbol{\mu},\boldsymbol{\Sigma}) +
\lambda (\sum_{k=1}^{K} \pi_k -1)$$

which yields

$$0=\sum_{n=1}^{N}\gamma(z_{nk}) + \lambda.$$

The result is $\lambda = -N$, yielding

$$\pi_k = \frac{N_k}{N}$$

which is kind of what you'd like to end up with, i.e., the mixing coefficient should resemble the weighted fraction of points assigned to a cluster.

This procedure leaves us with three equations: one each for $\boldsymbol{\mu}_k$, $\boldsymbol{\Sigma}_k$, and $\pi_k$. They have complicated interdependencies, basically through the responsibilities. So despite their apparent simplicity, there is no closed form solution. But it is a simple problem for the **expectation-maximization method**.

The EM method, in this case, goes as follows. 
 - Initial values of the means, covariances, and mixing coefficients are chosen. 
 - In the expectation step, the current values of these parameters are used to evaluate the posterior probability, i.e., the responsibilities $\gamma(z_k)$. 
 - In the maximization step, those responsibilities are used to estimate new values of $\boldsymbol{\mu}_k$, $\boldsymbol{\Sigma}_k$, and $\pi_k$. 
 
The process is repeated until converged.

It can be shown that each update to the parameters (i.e., one set of E and M) is guaranteed to increase the log likelihood function. For a proof of this, see Section 9.4 in Bishop. In practice, it is determined to be converged when the difference in the log likelihood function has dropped below some threshold.  In this way it is similar to $\chi^2$ fitting.  In fact, EM is a frequentist method, since the goal is to obtain the best fitting values of $\mu$, $\sigma$, and $\pi$, rather than confidence intervals for same.  

Gaussian mixture models can take some time to converge compared with k-means, but that is perhaps to be expected given that k-means yields only the position of the clusters and cluster membership, while GMM yields that plus the widths of the clusters.  

So researchers can use kmeans to find good starting points for the GMM.  The covariance matrices can be initialized to the covariances of the clusters found by the K-means algorithm, and the mixing coefficients can be set to the fractions of points in each k-means cluster.

### EM for GMM - Summary


To summarize, the EM method for Gaussian Mixtures is outlined below:

1. Initialize the means $\boldsymbol{\mu}_k$, covariances $\boldsymbol{\Sigma}_k$, and mixing coefficients $\pi_k$, and evaluate the initial value of the log likelihood:

 $$\ln p(\mathbf{X}|\boldsymbol{\pi},\boldsymbol{\mu},\mathbf{\Sigma}) =
 \sum_{n-1}^{N} \ln \left[\sum_{k=1}^{K} \pi_K
 \mathcal{N}(x_n|\mu_k,E_k)\right ].$$

2. The E step. Evaluate the responsibilities using the current parameter values

 $$\gamma(z_{nk}) =
 \frac{\pi_k\mathcal{N}(\mathbf{x}_n|\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k)}{\sum_{j=1}^{K}
 \pi_j
 \mathcal{N}(\mathbf{x}_n|\boldsymbol{\mu}_j,\boldsymbol{\Sigma}_j)}.$$

3. The M step. Re-estimate the parameter values using the current responsibilities.  Note that the vector multiplication in the equation for $\boldsymbol{\Sigma}_k^{new}$ below is the outer product, not the inner product.

 $$\boldsymbol{\mu}_k^{new} =\frac{1}{N_k} \sum_{n=1}^N \gamma(z_{nk})
 \mathbf{x}_n$$

 $$\boldsymbol{\Sigma}_k^{new} = \frac{1}{N_k} \sum_{n=1}^{N} \gamma(z_{nk})
 (\mathbf{x}_n - \boldsymbol{\mu}_k^{new}) (\mathbf{x}_n
 -\boldsymbol{\mu}_k^{new})^T$$

 $$\pi_k^{new} = \frac{N_k}{N}$$

 where

 $$N_k=\sum_{n=1} \gamma(z_{nk})$$

4. Evaluate the log likelihood again. Compare with the previous one and check for convergence. If not yet converged, try another step.

Finally, I emphasize again that the EM method seems very general. Bishop describes a proof that each step of the EM will invariably increase the log likelihood. He also discusses the relationship to K-means, describes using a mixture of Bernoulli distributions, and applies it to linear regression. The interested student is urged to refer to Bishop for more information.

[Here's an example of GMM.](https://www.youtube.com/watch?v=B36fzChfyGU)



## How many Gaussians?  AIC and BIC

As you will recall, histograms and KDE included a parameter that had to be specified ahead of time: the width $h$.  

Here, we have another parameter that has to be specified: the number of gaussians.  How many are needed?

We can use some quite general concepts to determine this.  This discussion follows Ivezic 4.3.2 and 5.4.3.

First, it is clear that if you measure the maximum likelihood for several models, the model that produces the largest value is the best description of  data.  

However, this may not be the best model overall especially when models have different numbers of free parameters.  

So there needs to be a scoring system that will consider _both the likelihood and the model complexity_.  

A popular general classical method for model comparison is the **Aikake Information Criterion (AIC)**.  The AIC is defined as 

$$AIC = -2 \ln (L^0(M)) + 2 k + \frac{2k(k+1)}{N-k-1},$$

where $k$ is the number of model parameters and $N$ is the number of data points.  The AIC is related to methods based on bias-variance trade off and can be derived using information theory, apparently.

Under the assumption of normality, the first term is equal to the model's $\chi^2$ (plus or minus a constant). The second term penalizes by the number of parameters.  For example, for every additional gaussian in a GMM, three more parameters are needed. When multiple models are compared, the one with the lowest value of AIC is the one to select.  


In the Bayesian framework, a closely related parameter, the AIC, is available.  It is computed as:

$$ BIC = -2 \ln [L^0(M)]+ k \ln N$$

where $k$ is the number of model parameters, and $N$ is the number of data points.  

These parameters are simply guidelines, and may not hold up in all cases.  

It turns out that the GMM program above will output the AIC and BIC.  Let's run the program for the data above using different numbers of clusters.

In [ ]:
test_numcomponents=np.arange(6)+1
print test_numcomponents

AIC_result=np.zeros(6)
BIC_result=np.zeros(6)

for i in range(6):
    gmm_temp=GMM(n_components=test_numcomponents[i])
    result=gmm_temp.fit(data)
    AIC_result[i]=result.aic(data)
    BIC_result[i]=result.bic(data)
    

In [ ]:
    
plt.plot(test_numcomponents,AIC_result)
plt.plot(test_numcomponents,BIC_result)

### Caveats

Ivezic describes several caveats users of the GMM might want to keep in mind.

The first is that the specific clusters identified may or may not correspond to physically coherent groups. This is because groups in nature are not very likely to form nice gaussian distributions in conveniently measured variables. An example is given in Figure 6.6 from Ivezic, below.

![Ivezic Figure 6.6](http://www.astroml.org/_images/fig_EM_metallicity_1.png)

The take away message is that it may be better to use the cluster results as a smooth estimate of the density, an indicator of interesting phenomena. As pointed out by Ivezic, the mixture method approaches the flexibility of nonparametric density estimation when the number of clusters is large.

Ivezic also points out that _gaussian mixture models can perform poorly in the presence of background_. One way to get around this may be to fit with many components, then divide the resulting clusters into data and background based on density of points.



## Extreme Deconvolution: GMM with Errors

So far, the data sets we have investigated have no associated uncertainties. What can we do when there is uncertainty present? GMM on data that have measurement errors is called "extreme deconvolution" (XD); see Ivezic for references. This discussion follows Ivezic 6.3.3.

We will imagine that each data point $\mathbf{w_i}$ are related to the true values $\mathbf{v_i}$ through

$$\mathbf{w_i}=\mathbf{R_i}\mathbf{v_i}+\epsilon_i,$$

where $\mathbf{R}_i$ is called the projection matrix, which looks like it applies principally when there is missing data, and the noise $\epsilon_i$ is assumed to have been drawn from a Gaussian with zero mean and variance $\mathbf{S}_i$. The matrices $\mathbf{R}_i$ and $\mathbf{S}_i$ are assumed to be known, as they are based on the data.

The goal is to find, as above, the parameters $\boldsymbol{\mu}_j$, $\mathbf{\Sigma}_j$, and $\pi_j$. (Note the typos in Ivezic section 6.3.3 - the index $i$ refers to the data points, and the index $j$ refers to the clusters. In addition, the noisy values should be referred to as $\mathbf{w}_i$ rather than $\mathbf{x}_i$ as in Equation 6.20. There are other typos than this, best to see the paper describing the algorithm, [Bovy et al. 2011.](http://adsabs.harvard.edu/abs/2011AnApS...5.1657B))  A review of this paper is found [here](http://seminar.ouml.org/applications/review-of-think-outside-the-color-box-probabilistic-target-selection-and-the-sdss-xdqso-quasar-targeting-catalog/).

In this case, the expectation step looks like:

$$\gamma_{ij}=\frac{\pi_j \mathcal{N}(\mathbf{w}_i|\mathbf{R_i}
\boldsymbol{\mu_j},\mathbf{T}_{ij})}{\sum_j
\pi_j\mathcal{N}(\mathbf{w}_i | \mathbf{R}_i
\boldsymbol{\mu}_j,\mathbf{T}_{ij})}$$

$$\mathbf{b}_{ij}=\boldsymbol{\mu}_j+\boldsymbol{\Sigma}_j
\mathbf{R}_i^T \mathbf{T}_{ij}^{-1} (\mathbf{w_i}-\mathbf{R}_i
\boldsymbol{\mu}_j)$$

$$\mathbf{B}_{ij} = \boldsymbol{\Sigma}_j-\boldsymbol{\Sigma}_j\mathbf{R}_i^T
\mathbf{T}^{-1}_{ij} \boldsymbol{\Sigma}_j$$

where $$\mathbf{T}_{ij}=\mathbf{R}_i \boldsymbol{\Sigma}_j
\mathbf{R}_i^T + \mathbf{S}_i.$$

The maximization step looks like:

$$\pi_j^{new} = \frac{N_j}{N}$$

$$\boldsymbol{\mu}_j^{new} = \frac{1}{N_j} \sum_i \gamma_{ij} b_{ij}$$

$$\boldsymbol{\Sigma}_j^{new}= \frac{1}{N_j} \sum_i
\gamma_{ij}[(\boldsymbol{\mu}_j -
\mathbf{b}_{ij})(\boldsymbol{\mu}_j-\mathbf{b}^T_{ij})+\mathbf{B}_{ij}].$$

Ivezic Figure 6.11 shows an implementation of the extreme deconvolution to simulated data.  The top left panel shows the distribution with small errors, while the right distribution shows the same with large errors.  The bottom panels show the densities derived from the noisy sample; as you can see, they closely resemble the distribution with small errors.

![Ivezic Figure 6.11](http://www.astroml.org/_images/fig_XD_example_1.png)

## Hierachical Clustering

In all the methods discussed above, the number of clusters is specified ahead of time.  While there are formal and informal methods to determine how many  clusters provides the best model, this procedure seems less than convenient.

Hierarchical clustering avoids this requirement by finding clusters at all scales.  For example, the data can be partitioned into $N$ clusters, one for each point in the data set.  Two clusters are found not to differ by much, so they are joined together, resulting in $N-1$ clusters.  This procedure is continued until all the points are in one cluster, or the differences between some $m$ clusters is deemed sufficient.

Hierarchical clustering can be approached as a bottom-up (_agglomerative_) procedure, where data is progressively joined together, as described above, or as a top-down (_divisive_) procedure, where the data is progressively subdivided.  Ivezic considers the agglomerative application.

This discussion follows Ivezic 6.4.5, but also draws upon Nathalie Thibert's undergraduate honor's thesis (University of Western Ontario).



The procedure goes as follows:
- At each step, the nearest pair of clusters is merged.

What defines the nearest pair of clusters?  There are many options, including 

$$d_{min}(C_k,C_{k^\prime}) = min \lVert x-x^\prime \rVert, x\in C_k, x^\prime \in C_{k^\prime},$$

$$d_{max}(C_k,C_{k^\prime}) = max \lVert x-x^\prime \rVert, x\in C_k, x^\prime \in C_{k^\prime},$$

$$d_{avg}(C_k,C_{k^\prime}) = \frac{1}{N_k N_{k^\prime}} \sum_{x\in C_k} \sum_{x^\prime \in C_{k^\prime}} \lVert x-x^\prime \rVert,$$

$$d_{cen}(C_k,C_{k^\prime}) = \lVert \mu_k -\mu_{k^\prime} \rVert,$$

where $x$ and $x^\prime$ are the points in cluster $C_{k}$ and $C_{k^\prime}$, respectively.  Likewise, $N_k$ and $N_{k^\prime}$ are the number of points in each cluster, and $\mu_k$ and $\mu_{k^\prime}$ are the centroids of the clusters.



- Using the distance $d_{min}$ results in hierarchical clustering known as a "minimal spanning tree".  This will produce clusters with extended chains of points.
- Using $d_{max}$ will produce compact clusters.
- The other two produce clusters with behavior in between these extremes.

This procedure is continued until only one cluster remains.  The result of cluster merges is called a **dendogram**.  (note that this image shows a divisive procedure, whereas we are talking about the inverse, the agglomerative procedure.)

![](https://onlinecourses.science.psu.edu/stat505/sites/onlinecourses.science.psu.edu.stat505/files/lesson12/dendogram_example.gif)

Next, you interpret the dendogram.  

Generally speaking, you choose a threshold distance and apply a horizontal cut there. This threshold distance is the maximum distance allowed between points in a cluster.  Each unique branch below the cut becomes a "flat cluster".     The farther down you go, the more similar the objects in a cluster are to one another.  

You don't necessarily have to cut along a horizontal line; you can cut some branches higher than others. See the example.

Ivezic Figure 6.15 computes a hierachical clustering of the SDSS "Great Wall" using the $d_{min}$ criterion.   The extended chains of points trace the large-scale structure within the data.   

![Ivezic Figure 6.15](http://www.astroml.org/_images/fig_great_wall_MST_1.png)

According to Ivezic, one can post-process these results by, for example, sorting the links by increasing length, then taking those with lengths less than some threshold as the clusters.   For a "single-linkage hierarchical clustering", this is known as "friends-of-friends" clustering.

A minimal spanning tree is naively $\mathcal{O} (N^3)$ to compute.  However, apparently there is an algorithm that will do the job in approximate $\mathcal{O}(N \log N)$.